# Chapter 44: Upgrading Next.js Versions

Keeping your Next.js application current is essential for security, performance, and access to the latest features. However, upgrades can introduce breaking changes, deprecations, and subtle behavioral differences that affect production stability. This chapter provides battle-tested strategies for planning, executing, and validating Next.js version upgrades—whether you're jumping one minor version or migrating across major releases with significant architectural shifts.

By the end of this chapter, you'll master upgrade planning methodologies, handle breaking changes systematically, leverage codemods for automated refactoring, manage dependency compatibility, implement comprehensive upgrade testing strategies, prepare rollback procedures for failed deployments, and establish workflows for staying current without disrupting development velocity.

## 44.1 Upgrade Strategies

Choosing the right approach to upgrading minimizes risk and downtime while ensuring your team can adapt to changes incrementally.

### Semantic Versioning Understanding

Next.js follows semantic versioning (semver) principles that dictate risk levels:

```typescript
// Version format: MAJOR.MINOR.PATCH
// 14.2.3 → Major: 14, Minor: 2, Patch: 3

/**
 * Patch releases (14.2.0 → 14.2.1)
 * - Bug fixes only
 * - Zero breaking changes
 * - Safe to auto-update
 */
// .github/workflows/auto-update.yml
name: Patch Auto-Update
on:
  schedule:
    - cron: '0 2 * * 1' # Weekly on Monday
jobs:
  update:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-node@v4
        with:
          node-version: '20'
      - run: npm update next
      - run: npm test
      - name: Create PR
        uses: peter-evans/create-pull-request@v5
        with:
          title: 'chore: bump Next.js patch version'
          branch: 'auto/next-patch'

/**
 * Minor releases (14.1.x → 14.2.0)
 * - New features added
 * - Deprecations may be introduced (but old APIs still work)
 * - Low to medium risk
 */
// Upgrade approach: Staged rollout with feature flags
// lib/features.ts
export const features = {
  // New features from minor upgrades gated behind flags
  experimentalPartialPrerendering: process.env.NEXT_EXPERIMENTAL_PPR === 'true',
  newDevOverlay: process.env.NEXT_NEW_OVERLAY === 'true',
};

/**
 * Major releases (14.x → 15.0.0)
 * - Breaking changes expected
 * - APIs may be removed or significantly altered
 * - High risk, requires comprehensive testing
 */
// Upgrade approach: Dedicated migration sprint with parallel branches
```

### Incremental vs. Big-Bang Upgrades

Choose your strategy based on application complexity and team capacity:

```typescript
// Strategy 1: Incremental Canary Deployment
// Deploy new version to small traffic percentage first

// vercel.json for canary deployment
{
  "version": 2,
  "routes": [
    {
      "src": "/(.*)",
      "dest": "/$1",
      "headers": {
        "x-nextjs-version": "15.0.0"
      }
    }
  ],
  "rewrites": [
    {
      "source": "/api/:path*",
      "destination": "https://canary-api.yoursite.com/api/:path*"
    }
  ]
}

// Strategy 2: Parallel Application Instances
// Run old and new versions simultaneously with routing layer

// middleware.ts for version routing
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  // Route beta users to new version
  const isBeta = request.cookies.get('beta-opt-in')?.value === 'true';
  const isNewVersion = request.nextUrl.pathname.startsWith('/v2');
  
  if (isBeta && !isNewVersion) {
    const url = request.nextUrl.clone();
    url.pathname = `/v2${request.nextUrl.pathname}`;
    return NextResponse.rewrite(url);
  }
  
  return NextResponse.next();
}

// Strategy 3: Blue-Green Deployment
// Instant switchover with instant rollback capability

// scripts/deploy-blue-green.sh
#!/bin/bash
BLUE_URL="https://app-blue.vercel.app"
GREEN_URL="https://app-green.vercel.app"
DOMAIN="yourdomain.com"

# Deploy to green environment
echo "Deploying to green environment..."
vercel --target=production --name=app-green

# Health check green
if curl -sf "$GREEN_URL/api/health"; then
  echo "Green health check passed"
  
  # Switch traffic
  vercel alias set "$GREEN_URL" "$DOMAIN"
  
  # Monitor for 10 minutes
  sleep 600
  
  # Verify no errors, then deploy blue
  ERROR_RATE=$(curl -s "$GREEN_URL/api/metrics" | jq '.errorRate')
  if (( $(echo "$ERROR_RATE < 0.01" | bc -l) )); then
    echo "Upgrade successful, updating blue environment..."
    vercel --target=production --name=app-blue
  else
    echo "Error rate too high, rolling back..."
    vercel alias set "$BLUE_URL" "$DOMAIN"
    exit 1
  fi
else
  echo "Green health check failed"
  exit 1
fi
```

### Upgrade Planning Checklist

Systematic preparation prevents production incidents:

```typescript
// scripts/upgrade-checklist.ts
import { execSync } from 'child_process';
import fs from 'fs';
import path from 'path';

interface UpgradeCheck {
  category: string;
  checks: {
    name: string;
    command?: string;
    manual?: boolean;
    description: string;
  }[];
}

const upgradePlan: UpgradeCheck[] = [
  {
    category: 'Pre-Upgrade Analysis',
    checks: [
      {
        name: 'Current Version Lock',
        command: 'npm list next --depth=0',
        description: 'Document current version for rollback reference',
      },
      {
        name: 'Dependency Tree Analysis',
        command: 'npm ls --depth=0',
        description: 'Identify peer dependencies that may conflict',
      },
      {
        name: 'Deprecated APIs Audit',
        manual: true,
        description: 'Search codebase for deprecated patterns (next/head, getInitialProps, etc.)',
      },
      {
        name: 'Bundle Size Baseline',
        command: 'npm run build && npm run analyze',
        description: 'Record current bundle sizes to detect regressions',
      },
    ],
  },
  {
    category: 'Staging Validation',
    checks: [
      {
        name: 'Build Verification',
        command: 'npm run build',
        description: 'Ensure production build succeeds without errors',
      },
      {
        name: 'TypeScript Compilation',
        command: 'npx tsc --noEmit',
        description: 'Check for type errors introduced by new types',
      },
      {
        name: 'Test Suite Pass',
        command: 'npm test',
        description: 'All existing tests must pass',
      },
      {
        name: 'E2E Critical Path',
        command: 'npm run test:e2e: smoke',
        description: 'Verify login, checkout, critical user flows',
      },
    ],
  },
  {
    category: 'Production Readiness',
    checks: [
      {
        name: 'Rollback Plan',
        manual: true,
        description: 'Document exact steps and timing for rollback',
      },
      {
        name: 'Monitoring Alerts',
        manual: true,
        description: 'Verify error tracking and performance monitoring active',
      },
      {
        name: 'Database Compatibility',
        manual: true,
        description: 'Confirm no schema changes required for new version',
      },
      {
        name: 'CDN Cache Purge Strategy',
        manual: true,
        description: 'Plan for cache invalidation during deployment',
      },
    ],
  },
];

async function runChecklist(targetVersion: string) {
  console.log(`\n🚀 Preparing upgrade to Next.js ${targetVersion}\n`);
  
  const results: { passed: string[]; failed: string[]; manual: string[] } = {
    passed: [],
    failed: [],
    manual: [],
  };

  for (const category of upgradePlan) {
    console.log(`\n📋 ${category.category}`);
    console.log('='.repeat(50));

    for (const check of category.checks) {
      process.stdout.write(`  ⏳ ${check.name}... `);

      if (check.manual) {
        results.manual.push(`${category.category}: ${check.name} - ${check.description}`);
        console.log('⚠️  MANUAL');
        continue;
      }

      try {
        if (check.command) {
          execSync(check.command, { stdio: 'pipe', encoding: 'utf-8' });
        }
        results.passed.push(check.name);
        console.log('✅ PASS');
      } catch (error) {
        results.failed.push(`${check.name}: ${check.description}`);
        console.log('❌ FAIL');
      }
    }
  }

  // Generate report
  const report = {
    targetVersion,
    timestamp: new Date().toISOString(),
    summary: {
      total: results.passed.length + results.failed.length + results.manual.length,
      passed: results.passed.length,
      failed: results.failed.length,
      manual: results.manual.length,
    },
    details: results,
  };

  fs.writeFileSync(
    path.join(process.cwd(), 'upgrade-report.json'),
    JSON.stringify(report, null, 2)
  );

  console.log('\n' + '='.repeat(50));
  console.log('Upgrade Readiness Report');
  console.log('='.repeat(50));
  console.log(`Passed: ${results.passed.length} | Failed: ${results.failed.length} | Manual: ${results.manual.length}`);

  if (results.failed.length > 0) {
    console.log('\n❌ Failed Checks (blocking):');
    results.failed.forEach(f => console.log(`   - ${f}`));
    process.exit(1);
  }

  if (results.manual.length > 0) {
    console.log('\n⚠️  Manual Checks Required:');
    results.manual.forEach(m => console.log(`   - ${m}`));
  }

  console.log('\n✅ Automated checks passed! Proceed with manual verification.');
}

// Usage: npx ts-node scripts/upgrade-checklist.ts 15.0.0
runChecklist(process.argv[2]);
```

## 44.2 Breaking Changes

Major version upgrades introduce intentional breaking changes that require code modifications.

### Common Breaking Changes by Version

Key patterns to watch for in recent upgrades:

```typescript
// Next.js 13 → 14 Breaking Changes
// =================================

// 1. Node.js Version Requirements
// Before: Node 16.8+
// After: Node 18.17+
// Check: package.json engines field
{
  "engines": {
    "node": ">=18.17.0",
    "npm": ">=9.0.0"
  }
}

// 2. WASM Loading in Edge Runtime
// Before: Direct instantiation
const wasmModule = await WebAssembly.compile(buffer);

// After: Explicit initialization required
import { compile } from 'next/wasm';
const wasmModule = await compile(buffer);

// 3. Image Optimization API Changes
// Before: Unoptimized images in dev by default
// After: Always optimized unless explicitly disabled
// next.config.js
module.exports = {
  images: {
    unoptimized: process.env.NODE_ENV === 'development',
  },
};

// Next.js 14 → 15 Breaking Changes (Hypothetical/Future-Proofing)
// ================================================================

// 1. Cache Behavior Changes
// Before: fetch cached by default
fetch('https://api.example.com/data');

// After: Explicit cache configuration required
fetch('https://api.example.com/data', {
  cache: 'force-cache', // or 'no-store', 'revalidate'
  next: { revalidate: 3600 }
});

// 2. Metadata API Strictness
// Before: Certain meta tags allowed in both layout and page
// After: Strict hierarchical enforcement
// app/layout.tsx - Only global metadata
export const metadata = {
  title: {
    template: '%s | My Site',
    default: 'My Site',
  },
};

// app/page.tsx - Page specific only
export const metadata = {
  title: 'Home Page', // Must fit template
  // description must be here or in layout, not both with different values
};

// 3. Dynamic API Changes
// Before: Dynamic APIs synchronous
const pathname = usePathname();

// After: Async dynamic APIs (React 19 / Next 15+)
const pathname = await usePathname();
```

### Breaking Change Detection Script

Automate detection of problematic patterns:

```typescript
// scripts/detect-breaking-patterns.ts
import { glob } from 'glob';
import fs from 'fs/promises';
import path from 'path';

interface BreakingPattern {
  id: string;
  name: string;
  version: string;
  pattern: RegExp;
  suggestion: string;
  severity: 'error' | 'warning';
}

const breakingPatterns: BreakingPattern[] = [
  {
    id: 'NEXT-001',
    name: 'getInitialProps Usage',
    version: '12+',
    pattern: /static\s+async\s+getInitialProps|getInitialProps\s*\(/g,
    suggestion: 'Migrate to getStaticProps, getServerSideProps, or data fetching in Server Components',
    severity: 'error',
  },
  {
    id: 'NEXT-002',
    name: 'next/head in App Router',
    version: '13+',
    pattern: /import.*next\/head.*from.*next\/head/,
    suggestion: 'Use Metadata API instead of next/head in App Router',
    severity: 'error',
  },
  {
    id: 'NEXT-003',
    name: 'Deprecated Image Layout',
    version: '13+',
    pattern: /layout\s*=\s*["'](fill|fixed|intrinsic|responsive)["']/,
    suggestion: 'Use style prop or className with fill and sizes instead of layout prop',
    severity: 'warning',
  },
  {
    id: 'NEXT-004',
    name: 'URL Object in Link',
    version: '13+',
    pattern: /href\s*=\s*\{\s*\{ pathname/,
    suggestion: 'Use URL strings with template literals instead of URL objects in next/link',
    severity: 'warning',
  },
  {
    id: 'NEXT-005',
    name: 'Server Components Client API',
    version: '14+',
    pattern: /['"]use client['"].*\n.*usePathname|useSearchParams|useRouter/,
    suggestion: 'Ensure client hooks are only used in Client Components with "use client"',
    severity: 'error',
  },
  {
    id: 'NEXT-006',
    name: 'Api Routes Body Parsing',
    version: '15+',
    pattern: /export\s+const\s+config\s*=\s*\{[^}]*api:\s*\{[^}]*bodyParser/,
    suggestion: 'Body parsing configuration has changed in Route Handlers',
    severity: 'warning',
  },
];

async function scanFile(filePath: string): Promise<{ file: string; issues: any[] }> {
  const content = await fs.readFile(filePath, 'utf-8');
  const issues = [];

  for (const pattern of breakingPatterns) {
    const matches = content.match(pattern.pattern);
    if (matches) {
      // Find line numbers
      const lines = content.split('\n');
      matches.forEach((match) => {
        const lineIndex = lines.findIndex(line => line.includes(match));
        issues.push({
          ...pattern,
          line: lineIndex + 1,
          match: match.substring(0, 50),
        });
      });
    }
  }

  return { file: filePath, issues };
}

async function runDetection() {
  const files = await glob('**/*.{ts,tsx,js,jsx}', {
    ignore: ['node_modules/**', '.next/**', 'out/**'],
  });

  console.log(`Scanning ${files.length} files for breaking changes...\n`);

  const results = await Promise.all(files.map(scanFile));
  const filesWithIssues = results.filter(r => r.issues.length > 0);

  let errorCount = 0;
  let warningCount = 0;

  for (const result of filesWithIssues) {
    console.log(`\n📄 ${path.relative(process.cwd(), result.file)}`);
    
    const grouped = result.issues.reduce((acc, issue) => {
      acc[issue.id] = acc[issue.id] || { ...issue, count: 0 };
      acc[issue.id].count++;
      return acc;
    }, {});

    Object.values(grouped).forEach((issue: any) => {
      const icon = issue.severity === 'error' ? '❌' : '⚠️';
      console.log(`  ${icon} [${issue.id}] ${issue.name} (Line ${issue.line})`);
      console.log(`     ${issue.suggestion}`);
      
      if (issue.severity === 'error') errorCount++;
      else warningCount++;
    });
  }

  console.log('\n' + '='.repeat(50));
  console.log(`Scan complete: ${errorCount} errors, ${warningCount} warnings`);

  if (errorCount > 0) {
    console.log('\n❌ Fix all errors before upgrading');
    process.exit(1);
  } else if (warningCount > 0) {
    console.log('\n⚠️  Review warnings before upgrading');
  } else {
    console.log('\n✅ No breaking patterns detected');
  }
}

runDetection().catch(console.error);
```

## 44.3 Using Codemods

Codemods are automated transformation scripts that refactor code to accommodate API changes.

### Running Next.js Codemods

Official codemods provided by the Next.js team:

```bash
# Install codemod globally or use npx
npm install -g @next/codemod

# List available codemods
npx @next/codemod --list

# Common transformations:

# 1. Migrate from next/head to Metadata API (13+)
npx @next/codemod@latest metadata/app-dir .

# 2. Convert Image component to Next.js 13+ format
npx @next/codemod@latest next-image-to-legacy-image .

# 3. Convert to next/image import
npx @next/codemod@latest next-image-experimental .

# 4. Remove <a> tags from Link components (13+)
npx @next/codemod@latest new-link .

# 5. Migrate CRA to Next.js
npx @next/codemod@latest cra-to-next .

# 6. Add missing React imports (for React 17+ automatic runtime)
npx @next/codemod@latest add-missing-react-import .
```

### Custom Codemod Development

Write custom transformations for your specific patterns:

```typescript
// codemods/replace-fetch-pattern.ts
// jscodeshift transform for custom fetch pattern updates
import { API, FileInfo } from 'jscodeshift';

export default function transformer(file: FileInfo, api: API) {
  const j = api.jscodeshift;
  const root = j(file.source);

  // Pattern: Replace old data fetching with new pattern
  // Before: const data = await fetchData('/api/users')
  // After: const data = await fetch('/api/users').then(r => r.json())

  root
    .find(j.CallExpression, {
      callee: {
        name: 'fetchData',
      },
    })
    .replaceWith((path) => {
      const args = path.value.arguments;
      
      // Transform to standard fetch
      return j.callExpression(
        j.memberExpression(
          j.callExpression(j.identifier('fetch'), args),
          j.identifier('then')
        ),
        [
          j.arrowFunctionExpression(
            [j.identifier('r')],
            j.callExpression(
              j.memberExpression(j.identifier('r'), j.identifier('json')),
              []
            )
          ),
        ]
      );
    });

  // Remove old import
  root
    .find(j.ImportDeclaration, {
      source: { value: '@/lib/fetchData' },
    })
    .remove();

  return root.toSource();
}

// Running custom codemod
// jscodeshift -t codemods/replace-fetch-pattern.ts --parser=tsx --extensions=ts,tsx app/
```

### Safe Codemod Execution

Best practices for running automated transformations:

```typescript
// scripts/run-codemod-safely.ts
import { execSync } from 'child_process';
import fs from 'fs';
import path from 'path';

interface CodemodConfig {
  name: string;
  command: string;
  files: string[];
  backupDir: string;
}

async function runCodemodSafely(config: CodemodConfig) {
  const timestamp = new Date().toISOString().replace(/[:.]/g, '-');
  const backupPath = path.join(config.backupDir, `${config.name}-${timestamp}`);

  console.log(`🔧 Running codemod: ${config.name}`);

  // Step 1: Create backup
  console.log('  📦 Creating backup...');
  fs.mkdirSync(backupPath, { recursive: true });
  
  for (const file of config.files) {
    if (fs.existsSync(file)) {
      const dest = path.join(backupPath, file);
      fs.mkdirSync(path.dirname(dest), { recursive: true });
      fs.copyFileSync(file, dest);
    }
  }

  // Step 2: Run codemod
  console.log('  🔄 Executing transformation...');
  try {
    execSync(config.command, { stdio: 'inherit' });
  } catch (error) {
    console.error('  ❌ Codemod failed, restoring backup...');
    
    // Restore from backup
    for (const file of config.files) {
      const backupFile = path.join(backupPath, file);
      if (fs.existsSync(backupFile)) {
        fs.copyFileSync(backupFile, file);
      }
    }
    
    throw error;
  }

  // Step 3: Validation
  console.log('  ✅ Validating changes...');
  try {
    execSync('npm run type-check', { stdio: 'inherit' });
    execSync('npm run lint', { stdio: 'inherit' });
    execSync('npm run test:unit', { stdio: 'inherit' });
  } catch (error) {
    console.error('  ❌ Validation failed');
    console.log('  To restore: cp -r ${backupPath}/* .');
    throw error;
  }

  // Step 4: Cleanup backup on success
  console.log('  🧹 Cleaning up backup...');
  fs.rmSync(backupPath, { recursive: true, force: true });

  console.log(`  ✅ Codemod ${config.name} completed successfully`);
}

// Usage
const codemods: CodemodConfig[] = [
  {
    name: 'next-image-upgrade',
    command: 'npx @next/codemod@latest next-image-experimental ./app',
    files: ['app/**/*.{tsx,ts}', 'components/**/*.{tsx,ts}'],
    backupDir: './.codemod-backups',
  },
  {
    name: 'link-component-update',
    command: 'npx @next/codemod@latest new-link ./app',
    files: ['app/**/*.{tsx,ts}'],
    backupDir: './.codemod-backups',
  },
];

async function runAll() {
  for (const codemod of codemods) {
    await runCodemodSafely(codemod);
  }
  console.log('\n🎉 All codemods completed successfully');
}

runAll().catch((error) => {
  console.error('Codemod execution failed:', error);
  process.exit(1);
});
```

## 44.4 Dependency Updates

Coordinate Next.js upgrades with ecosystem dependency updates.

### Dependency Compatibility Matrix

Ensure all dependencies support your target Next.js version:

```typescript
// scripts/check-compatibility.ts
import { execSync } from 'child_process';
import fs from 'fs';

interface DependencyCheck {
  name: string;
  current: string;
  required: string;
  compatible: boolean;
  blocking: boolean;
}

async function checkCompatibility(targetNextVersion: string) {
  const packageJson = JSON.parse(fs.readFileSync('package.json', 'utf-8'));
  
  // Known peer dependency requirements by Next.js version
  const peerRequirements: Record<string, Record<string, string>> = {
    '14.0.0': {
      react: '^18.2.0',
      'react-dom': '^18.2.0',
      typescript: '^5.0.0',
      '@types/react': '^18.2.0',
      '@types/node': '^18.0.0',
      eslint: '^8.0.0',
      'eslint-config-next': '14.0.0',
    },
    '15.0.0': {
      react: '^19.0.0',
      'react-dom': '^19.0.0',
      typescript: '^5.3.0',
      '@types/react': '^18.2.0',
      '@types/node': '^20.0.0',
      eslint: '^8.0.0',
      'eslint-config-next': '15.0.0',
    },
  };

  const requirements = peerRequirements[targetNextVersion];
  if (!requirements) {
    console.log(`⚠️ No compatibility data for Next.js ${targetNextVersion}`);
    return;
  }

  const checks: DependencyCheck[] = [];
  const allDeps = {
    ...packageJson.dependencies,
    ...packageJson.devDependencies,
  };

  for (const [dep, requiredVersion] of Object.entries(requirements)) {
    const current = allDeps[dep];
    checks.push({
      name: dep,
      current: current || 'not installed',
      required: requiredVersion,
      compatible: current ? satisfiesRange(current, requiredVersion) : false,
      blocking: !current || !satisfiesRange(current, requiredVersion),
    });
  }

  // Check for known incompatible packages
  const knownIncompatible = [
    { name: 'next-auth', maxVersion: '4.24.0', nextVersion: '14.0.0' },
    { name: '@trpc/server', minVersion: '11.0.0', nextVersion: '15.0.0' },
  ];

  console.log('\n📋 Dependency Compatibility Check');
  console.log('='.repeat(60));

  let blockingCount = 0;

  for (const check of checks) {
    const status = check.compatible ? '✅' : '❌';
    const blocking = check.blocking ? ' [BLOCKING]' : '';
    console.log(`${status} ${check.name}: ${check.current} → ${check.required}${blocking}`);
    if (check.blocking) blockingCount++;
  }

  if (blockingCount > 0) {
    console.log(`\n❌ ${blockingCount} blocking dependencies found`);
    console.log('Update commands:');
    
    const updates = checks
      .filter(c => c.blocking)
      .map(c => `${c.name}@${c.required.replace('^', '').replace('~', '')}`)
      .join(' ');
    
    console.log(`npm install ${updates}`);
    process.exit(1);
  }

  console.log('\n✅ All dependencies compatible');
}

// Simple semver range check
function satisfiesRange(version: string, range: string): boolean {
  const cleanVersion = version.replace(/^\^|~/, '');
  const cleanRange = range.replace(/^\^|~/, '');
  
  // Simplified check - in production use semver library
  const [vMajor, vMinor] = cleanVersion.split('.').map(Number);
  const [rMajor, rMinor] = cleanRange.split('.').map(Number);
  
  if (range.startsWith('^')) {
    return vMajor === rMajor && (vMinor >= rMinor || vMajor > rMajor);
  }
  
  return vMajor >= rMajor && vMinor >= rMinor;
}

checkCompatibility(process.argv[2]);
```

### Lockfile Management

Handle package-lock.json and yarn.lock during upgrades:

```bash
# Strategy 1: Clean install approach
# Best for major upgrades with many dependency changes

# Backup existing lockfile
cp package-lock.json package-lock.json.backup

# Update Next.js
npm install next@latest

# Remove node_modules and lockfile for clean slate
rm -rf node_modules package-lock.json

# Reinstall with new versions
npm install

# If issues occur, restore backup
# mv package-lock.json.backup package-lock.json

# Strategy 2: Selective update (safer for minor/patch)
# Only updates Next.js and necessary peers
npm install next@latest react@latest react-dom@latest --save
npm install @types/react@latest @types/react-dom@latest --save-dev

# Strategy 3: Using npm-check-updates for batch updates
npx npm-check-updates --filter /next/ --upgrade
npm install
```

## 44.5 Testing Upgrades

Comprehensive testing prevents production regressions.

### Upgrade Testing Pyramid

Different test layers for upgrade validation:

```typescript
// 1. Static Analysis Tests
// __tests__/upgrade/static.test.ts
import { execSync } from 'child_process';

describe('Upgrade Static Checks', () => {
  test('TypeScript compilation succeeds', () => {
    expect(() => {
      execSync('npx tsc --noEmit', { stdio: 'pipe' });
    }).not.toThrow();
  });

  test('ESLint passes with Next.js config', () => {
    expect(() => {
      execSync('npx eslint . --ext .ts,.tsx', { stdio: 'pipe' });
    }).not.toThrow();
  });

  test('No duplicate React versions', () => {
    const output = execSync('npm ls react', { encoding: 'utf-8' });
    const versions = output.match(/react@\d+\.\d+\.\d+/g) || [];
    const uniqueVersions = [...new Set(versions)];
    expect(uniqueVersions.length).toBe(1);
  });
});

// 2. Unit Tests for API Changes
// __tests__/upgrade/api-compatibility.test.ts
import { render } from '@testing-library/react';
import Image from 'next/image';

describe('Next.js API Compatibility', () => {
  test('Image component renders with new props', () => {
    const { container } = render(
      <Image
        src="/test.jpg"
        alt="Test"
        width={100}
        height={100}
        priority
        quality={75}
      />
    );
    expect(container.querySelector('img')).toBeTruthy();
  });

  test('Metadata API exports work', () => {
    // Dynamic import to avoid build-time execution
    const loadMetadata = async () => {
      const mod = await import('@/app/layout');
      return mod.metadata;
    };
    
    expect(loadMetadata()).resolves.toBeDefined();
  });
});

// 3. Integration Tests
// __tests__/upgrade/routing.test.ts
import { createRequest, createResponse } from 'node-mocks-http';

describe('Routing Behavior', () => {
  test('Middleware executes without errors', async () => {
    const { middleware } = await import('@/middleware');
    const req = createRequest({ url: '/protected' });
    const res = createResponse();
    
    const result = await middleware(req as any);
    expect(result).toBeDefined();
  });

  test('Route Handlers accept requests', async () => {
    const response = await fetch('http://localhost:3000/api/health');
    expect(response.status).toBe(200);
  });
});

// 4. Visual Regression Tests
// playwright.config.ts
import { defineConfig, devices } from '@playwright/test';

export default defineConfig({
  testDir: './e2e/upgrade',
  fullyParallel: true,
  forbidOnly: !!process.env.CI,
  retries: process.env.CI ? 2 : 0,
  workers: process.env.CI ? 1 : undefined,
  reporter: 'html',
  use: {
    baseURL: 'http://localhost:3000',
    trace: 'on-first-retry',
    screenshot: 'only-on-failure',
  },
  projects: [
    {
      name: 'chromium',
      use: { ...devices['Desktop Chrome'] },
    },
  ],
});

// e2e/upgrade/visual.spec.ts
import { test, expect } from '@playwright/test';

test.describe('Visual Regression', () => {
  test('homepage matches baseline', async ({ page }) => {
    await page.goto('/');
    await expect(page).toHaveScreenshot('homepage.png', {
      maxDiffPixels: 100,
    });
  });

  test('dark mode toggle works', async ({ page }) => {
    await page.goto('/');
    await page.click('[data-testid="theme-toggle"]');
    await expect(page).toHaveScreenshot('homepage-dark.png');
  });
});
```

### Smoke Testing Suite

Quick validation that critical paths work post-upgrade:

```typescript
// scripts/smoke-test.ts
import { chromium, Browser, Page } from 'playwright';

interface SmokeTest {
  name: string;
  path: string;
  validator: (page: Page) => Promise<boolean>;
}

const smokeTests: SmokeTest[] = [
  {
    name: 'Homepage Load',
    path: '/',
    validator: async (page) => {
      const title = await page.title();
      return title.includes('Expected Site Name');
    },
  },
  {
    name: 'API Health Check',
    path: '/api/health',
    validator: async (page) => {
      const content = await page.textContent('body');
      return content?.includes('ok') || content?.includes('healthy');
    },
  },
  {
    name: 'Dynamic Route',
    path: '/dashboard',
    validator: async (page) => {
      // Check for common error indicators
      const has500 = await page.$('text=Internal Server Error');
      const has404 = await page.$('text=This page could not be found');
      return !has500 && !has404;
    },
  },
  {
    name: 'Image Optimization',
    path: '/',
    validator: async (page) => {
      const images = await page.$$('img');
      // Check that images have next/image attributes
      for (const img of images.slice(0, 3)) {
        const src = await img.getAttribute('src');
        if (src && src.includes('_next/image')) {
          return true;
        }
      }
      return false;
    },
  },
];

async function runSmokeTests() {
  console.log('🔥 Starting Smoke Tests\n');
  
  const browser = await chromium.launch();
  const results: { name: string; passed: boolean; error?: string }[] = [];

  try {
    for (const test of smokeTests) {
      process.stdout.write(`  ⏳ ${test.name}... `);
      
      const page = await browser.newPage();
      
      try {
        await page.goto(`http://localhost:3000${test.path}`, {
          waitUntil: 'networkidle',
          timeout: 30000,
        });

        const passed = await test.validator(page);
        
        if (passed) {
          console.log('✅ PASS');
          results.push({ name: test.name, passed: true });
        } else {
          console.log('❌ FAIL (validation)');
          results.push({ name: test.name, passed: false, error: 'Validation failed' });
        }
      } catch (error) {
        console.log('❌ FAIL (error)');
        results.push({ 
          name: test.name, 
          passed: false, 
          error: error instanceof Error ? error.message : 'Unknown error' 
        });
      } finally {
        await page.close();
      }
    }
  } finally {
    await browser.close();
  }

  console.log('\n' + '='.repeat(50));
  const passed = results.filter(r => r.passed).length;
  const failed = results.filter(r => !r.passed).length;
  
  console.log(`Results: ${passed} passed, ${failed} failed`);
  
  if (failed > 0) {
    console.log('\nFailed tests:');
    results.filter(r => !r.passed).forEach(r => {
      console.log(`  - ${r.name}: ${r.error}`);
    });
    process.exit(1);
  }

  console.log('\n✅ All smoke tests passed');
}

runSmokeTests();
```

## 44.6 Rollback Strategies

Prepare for rapid recovery when upgrades fail in production.

### Rollback Procedures

Multiple rollback strategies based on deployment method:

```bash
# Strategy 1: Vercel Instant Rollback
# Fastest method for Vercel deployments

# List recent deployments
vercel ls

# Identify the last stable deployment (production before upgrade)
vercel alias [PREVIOUS_DEPLOYMENT_URL] yourdomain.com

# Or use Vercel CLI rollback
vercel rollback

# Strategy 2: Docker Image Rollback
# For containerized deployments

# Tag current production image before deploy
docker tag myapp:latest myapp:backup-$(date +%Y%m%d)

# If rollback needed:
docker stop myapp-new
docker run -d --name myapp -p 3000:3000 myapp:backup-[DATE]

# Strategy 3: Git-Based Rollback
# Revert code and redeploy

# Create emergency rollback branch
git checkout -b emergency-rollback main

# Revert the upgrade commit
git revert [UPGRADE_COMMIT_HASH]

# Fast-forward main to rollback (if no new commits)
git checkout main
git reset --hard emergency-rollback
git push origin main --force-with-lease

# Or use revert without force push
git checkout main
git revert HEAD --no-edit
git push origin main
```

### Database Compatibility

Ensure database migrations are backward compatible:

```typescript
// migrations/0003_add_user_preferences.sql
-- Forward migration
ALTER TABLE users ADD COLUMN preferences JSONB DEFAULT '{}';

-- Rollback script (store in same file as comment or separate file)
-- ALTER TABLE users DROP COLUMN preferences;

// app/lib/db.ts
// Ensure new columns have defaults for rollback scenarios
export async function getUserWithPreferences(userId: string) {
  const user = await db.user.findUnique({
    where: { id: userId },
  });
  
  // Handle case where column might not exist during rollback
  return {
    ...user,
    preferences: user?.preferences || {},
  };
}
```

### Feature Flags for Gradual Rollback

Use feature flags to disable new functionality without full deployment rollback:

```typescript
// lib/features.ts
export const features = {
  // New functionality from upgrade wrapped in flags
  newCachingBehavior: process.env.ENABLE_NEW_CACHE === 'true',
  upgradedImageComponent: process.env.ENABLE_NEW_IMAGE !== 'false', // Default on
  experimentalServerActions: process.env.ENABLE_SERVER_ACTIONS === 'true',
};

// Emergency kill switch endpoint
// app/api/admin/kill-switch/route.ts
import { NextResponse } from 'next/server';

export async function POST(request: Request) {
  const { feature, enabled } = await request.json();
  
  // In production, update environment variable or feature flag service
  // This is a simplified example
  process.env[`ENABLE_${feature.toUpperCase()}`] = enabled ? 'true' : 'false';
  
  // Log for audit
  console.log(`Feature ${feature} ${enabled ? 'enabled' : 'disabled'} via kill switch`);
  
  return NextResponse.json({ success: true, feature, enabled });
}
```

## 44.7 Staying Current

Establish sustainable workflows for continuous updates.

### Automated Update Workflows

GitHub Actions for automated dependency management:

```yaml
# .github/workflows/nextjs-update-checker.yml
name: Check for Next.js Updates

on:
  schedule:
    - cron: '0 9 * * 1' # Every Monday at 9 AM
  workflow_dispatch:

jobs:
  check-update:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Check for Next.js updates
        id: check
        run: |
          CURRENT=$(cat package.json | jq -r '.dependencies.next')
          LATEST=$(npm view next version)
          echo "current=$CURRENT" >> $GITHUB_OUTPUT
          echo "latest=$LATEST" >> $GITHUB_OUTPUT
          
          if [ "$CURRENT" != "$LATEST" ]; then
            echo "update_available=true" >> $GITHUB_OUTPUT
          else
            echo "update_available=false" >> $GITHUB_OUTPUT
          fi
      
      - name: Create upgrade ticket
        if: steps.check.outputs.update_available == 'true'
        uses: actions/github-script@v7
        with:
          script: |
            const current = '${{ steps.check.outputs.current }}';
            const latest = '${{ steps.check.outputs.latest }}';
            
            await github.rest.issues.create({
              owner: context.repo.owner,
              repo: context.repo.repo,
              title: `Upgrade Next.js ${current} → ${latest}`,
              body: `A new version of Next.js is available.\n\n**Current:** ${current}\n**Latest:** ${latest}\n\n### Checklist:\n- [ ] Review changelog\n- [ ] Run breaking change detection\n- [ ] Update in staging environment\n- [ ] Run full test suite\n- [ ] Deploy to production`,
              labels: ['dependencies', 'nextjs', 'upgrade']
            });

      - name: Create automated PR (patch only)
        if: steps.check.outputs.update_available == 'true'
        uses: peter-evans/create-pull-request@v5
        with:
          title: 'chore: bump Next.js to ${{ steps.check.outputs.latest }}'
          branch: 'deps/nextjs-${{ steps.check.outputs.latest }}'
          commit-message: 'chore: bump Next.js to ${{ steps.check.outputs.latest }}'
          body: |
            Automated Next.js version bump.
            
            **Changes:** ${{ steps.check.outputs.current }} → ${{ steps.check.outputs.latest }}
            
            Please verify:
            - [ ] Build passes
            - [ ] Tests pass
            - [ ] No breaking changes affect our codebase
```

### Changelog Monitoring

Stay informed about upstream changes:

```typescript
// scripts/monitor-changelog.ts
import https from 'https';

interface Release {
  tag_name: string;
  body: string;
  published_at: string;
  html_url: string;
}

async function fetchReleases(): Promise<Release[]> {
  return new Promise((resolve, reject) => {
    const options = {
      hostname: 'api.github.com',
      path: '/repos/vercel/next.js/releases',
      headers: {
        'User-Agent': 'Next.js-Upgrade-Monitor',
      },
    };

    https.get(options, (res) => {
      let data = '';
      res.on('data', chunk => data += chunk);
      res.on('end', () => {
        try {
          resolve(JSON.parse(data));
        } catch (e) {
          reject(e);
        }
      });
    }).on('error', reject);
  });
}

function analyzeImpact(release: Release): 'high' | 'medium' | 'low' {
  const body = release.body.toLowerCase();
  
  if (body.includes('breaking change') || body.includes('deprecated')) {
    return 'high';
  }
  if (body.includes('bug fix') || body.includes('patch')) {
    return 'low';
  }
  return 'medium';
}

async function monitor() {
  const releases = await fetchReleases();
  const currentVersion = process.env.CURRENT_NEXT_VERSION || '14.0.0';
  
  console.log(`Monitoring Next.js releases (current: ${currentVersion})\n`);
  
  const relevantReleases = releases
    .filter(r => r.tag_name > `v${currentVersion}`)
    .slice(0, 5);

  for (const release of relevantReleases) {
    const impact = analyzeImpact(release);
    const icon = impact === 'high' ? '🔴' : impact === 'medium' ? '🟡' : '🟢';
    
    console.log(`${icon} ${release.tag_name} (${impact} impact)`);
    console.log(`   Published: ${new Date(release.published_at).toLocaleDateString()}`);
    console.log(`   URL: ${release.html_url}`);
    
    // Extract breaking changes section
    const breakingMatch = release.body.match(/## Breaking Changes([\s\S]*?)(?=##|$)/i);
    if (breakingMatch) {
      console.log(`   Breaking Changes: ${breakingMatch[1].substring(0, 200)}...`);
    }
    console.log();
  }
}

monitor().catch(console.error);
```

### Team Knowledge Management

Document upgrade decisions and patterns:

```markdown
<!-- docs/UPGRADE_LOG.md -->
# Next.js Upgrade Log

## 15.0.0 (Planned)
**Date:** TBD
**Lead:** @developer-name

### Motivation
- React 19 support required for concurrent features
- Performance improvements in App Router
- New caching APIs

### Risk Assessment
- **High:** Server Actions API changes affect 12 files
- **Medium:** Image component prop changes
- **Low:** Metadata API additions (new features only)

### Preparation
- [ ] Run codemods on feature branch
- [ ] Update React types
- [ ] Test server action error boundaries

### Rollback Criteria
- Error rate > 1% for 10 minutes
- Any payment flow failures
- Performance degradation > 20%

---

## 14.2.0 (Completed)
**Date:** 2024-01-15
**Lead:** @developer-name

### Changes Applied
- Turbopack enabled for dev (50% faster builds)
- Partial Prerendering experimental feature enabled
- Metadata API improvements

### Issues Encountered
- Image component `placeholder="blur"` required explicit import
  - Solution: Added import for blurDataURL

### Lessons Learned
- Always test image-heavy pages after image component updates
```

## Key Takeaways from Chapter 44

1. **Semantic Versioning Strategy**: Treat patch versions (14.2.1) as zero-risk auto-updates, minor versions (14.3.0) as low-risk feature additions requiring validation, and major versions (15.0.0) as high-risk breaking changes requiring migration sprints. Use canary deployments or blue-green strategies for major upgrades.

2. **Breaking Change Detection**: Before upgrading, run detection scripts to identify deprecated APIs like `getInitialProps`, `next/head` in App Router, or legacy Image component props. Use the upgrade checklist script to validate Node.js versions, dependency compatibility, and bundle size baselines.

3. **Codemod Automation**: Leverage official `@next/codemod` transforms for common migrations (metadata API, Image component, Link updates). Always backup files before running codemods, validate with TypeScript and tests post-transformation, and develop custom jscodeshift transforms for proprietary patterns in your codebase.

4. **Dependency Coordination**: Major Next.js releases often require React, TypeScript, and ESLint updates. Use compatibility matrix scripts to verify peer dependencies satisfy new requirements before attempting installation to avoid version conflicts and broken builds.

5. **Testing Pyramid**: Validate upgrades through static analysis (TypeScript, ESLint), unit tests (API compatibility), integration tests (routing, middleware), and visual regression tests (Playwright screenshots). Implement smoke tests that verify homepage loads, API health checks pass, images optimize correctly, and no 500/404 errors appear on critical paths.

6. **Rollback Preparedness**: Maintain deployment backups, use Vercel's instant alias switching or Docker image tagging for rapid recovery, implement feature kill switches for gradual disabling of new functionality, and ensure database migrations remain backward compatible during transition periods.

7. **Continuous Currentness**: Automate update detection with GitHub Actions that create upgrade tickets and PRs for patch versions, monitor the Next.js GitHub releases API for changelog analysis, and maintain an upgrade log documenting decisions, issues encountered, and rollback criteria for organizational learning.

## Coming Up Next

**Chapter 45: Building a Blog Platform**

With your Next.js application now running on the latest stable version, it's time to apply your knowledge to building real-world projects. In Chapter 45, we'll architect a complete blog platform from scratch—designing database schemas for content management, implementing authentication for authors, building rich text editing experiences with MDX, optimizing for SEO with dynamic metadata, setting up image optimization for media, and deploying with proper caching strategies. You'll integrate the patterns learned throughout this guide into a cohesive, production-ready application.